To use this notebook in <a href="https://www.tutorialspoint.com/google_colab/google_colab_executing_external_python_files.htm">Google Colab</a>, please download the main folder from <a href="https://github.com/maiziezhoulab/RNN_BrainMaturation/archive/master.zip">here</a> and start from [Set up your Colab](#Set-up-your-Colab)

Or else, start from [Train&Analysis](#Train&Analysis)

# Set up your Colab
## Install Tensorflow

In [ ]:
!/opt/bin/nvidia-smi
!pip install tensorflow-gpu==1.15.0

## Mount your Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive',force_remount=True)

## Authenticate and create the PyDrive client

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Change dir to the script

In [ ]:
path = "/path/to/your/RNN_BrainMaturation"#example:"/content/drive/My Drive/RNN_BrainMaturation-master"
os.chdir(path)
!pwd

# Train&Analysis
## GPU set up

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Train
### Set parameters

In [ ]:
model_dir = 'data/6tasks'

#Hyper parameters
hp = {'activation': 'softplus',
    'n_rnn': 256,
    'learning_rate': 0.001,
    'mix_rule': True,
    'l1_h': 0.,
    'use_separate_input': False,
    'mature_target_perf': 0.95,
    'mid_target_perf': 0.65,
    'early_target_perf': 0.35,}

# rule set and rule trains
ruleset='all_new'
rule_trains=['overlap','zero_gap','gap','odr','odrd','gap500',]

### Start Training

In [ ]:
from training.train import train

train(model_dir,
      seed=0,
      hp=hp,
      ruleset=ruleset,
      rule_trains=rule_trains,
      display_step=20)

## Analysis
### Set parameters

In [ ]:
model_dir = 'data/6tasks'
trial_list_odr_odrd = range(520960,628480+1,1280)
trial_list_antisacc = range(0,102400+1,1280)

### Load Hyper parameter and training log

In [ ]:
from utils import tools
hp = tools.load_hp(model_dir)
log = tools.load_log(model_dir)
trial_list_all = range(log['trials'][0],log['trials'][-1]+1,1280)

### Print basic information

In [ ]:
from analysis.PSTH_print_basic_info import print_basic_info
print_basic_info(hp,log,model_dir,smooth_growth=True,smooth_window=5)

### Compute H

In [ ]:
from analysis.PSTH_compute_H import gen_task_info
gen_task_info(hp,log,model_dir,['odr','odrd','overlap','zero_gap','gap']) #only generate task_info.pkl

In [ ]:
recompute = False
from analysis.PSTH_compute_H import compute_H
compute_H(hp,log,model_dir,rules=['odr','odrd'],trial_list=trial_list_odr_odrd,recompute=recompute,)
compute_H(hp,log,model_dir,rules=['overlap','zero_gap','gap'],trial_list=trial_list_antisacc,recompute=recompute,)

### Generate neuron information

In [ ]:
recompute = False
from analysis.PSTH_gen_neuron_info import generate_neuron_info
generate_neuron_info(hp,model_dir,epochs=['stim1','delay1'],trial_list=trial_list_odr_odrd,rules=['odr','odrd'],recompute=recompute)
generate_neuron_info(hp,model_dir,epochs=['delay2'],trial_list=trial_list_odr_odrd,rules=['odrd'],recompute=recompute)
generate_neuron_info(hp,model_dir,epochs=['stim1'],trial_list=trial_list_antisacc,rules=['overlap','zero_gap','gap'],recompute=recompute)

### Plot tuning curve

In [ ]:
from analysis.PSTH_tunning_analysis import tunning_analysis
tunning_analysis(hp,log,model_dir,'odr','stim1',trial_list_odr_odrd)
tunning_analysis(hp,log,model_dir,'odr','delay1',trial_list_odr_odrd)
tunning_analysis(hp,log,model_dir,'odrd','stim1',trial_list_odr_odrd)
tunning_analysis(hp,log,model_dir,'odrd','delay2',trial_list_odr_odrd)
tunning_analysis(hp,log,model_dir,'gap','stim1',trial_list_antisacc)
tunning_analysis(hp,log,model_dir,'overlap','stim1',trial_list_antisacc)
tunning_analysis(hp,log,model_dir,'zero_gap','stim1',trial_list_antisacc)

### Plot PSTH

In [ ]:
from analysis.PSTH_plot_PSTH import plot_PSTH
plot_PSTH(hp, log, model_dir, 'odr', 'stim1', trial_list_odr_odrd, plot_oppo_dir = False)
plot_PSTH(hp, log, model_dir, 'odr', 'delay1', trial_list_odr_odrd, plot_oppo_dir = False)
plot_PSTH(hp, log, model_dir, 'odrd', 'stim1', trial_list_odr_odrd, plot_oppo_dir = True)
plot_PSTH(hp, log, model_dir, 'odrd', 'delay1', trial_list_odr_odrd, plot_oppo_dir = True)
plot_PSTH(hp, log, model_dir, 'odrd', 'delay2', trial_list_odr_odrd, plot_oppo_dir = True)

plot_PSTH(hp, log, model_dir, 'gap', 'stim1', trial_list_antisacc, plot_oppo_dir = False)
plot_PSTH(hp, log, model_dir, 'zero_gap', 'stim1', trial_list_antisacc, plot_oppo_dir = False)
plot_PSTH(hp, log, model_dir, 'overlap', 'stim1', trial_list_antisacc, plot_oppo_dir = False)

### Plot the growth of the epoch mean value

In [ ]:
from analysis.PSTH_gen_PSTH_log import gen_PSTH_log
from analysis.PSTH_plot_epoch_mean_growth import plot_epoch_mean_growth

PSTH_log = gen_PSTH_log(hp,trial_list_all,model_dir,rule = 'odr',seltive_epoch = 'stim1')
plot_epoch_mean_growth(hp,log, trial_list_all, model_dir, rule = 'odr', seltive_epoch = 'stim1', analy_epoch = 'stim1',PSTH_log = PSTH_log)
plot_epoch_mean_growth(hp,log, trial_list_all, model_dir, rule = 'odr', seltive_epoch = 'stim1', analy_epoch = 'delay1',PSTH_log = PSTH_log)

plot_epoch_mean_growth(hp,log, trial_list_all, model_dir, rule = 'odr', seltive_epoch = 'delay1', analy_epoch = 'delay1',)

PSTH_log = gen_PSTH_log(hp,trial_list_all,model_dir,rule = 'odrd',seltive_epoch = 'stim1')
plot_epoch_mean_growth(hp,log, trial_list_all, model_dir, rule = 'odrd', seltive_epoch = 'stim1', analy_epoch = 'stim1',PSTH_log = PSTH_log)
plot_epoch_mean_growth(hp,log, trial_list_all, model_dir, rule = 'odrd', seltive_epoch = 'stim1', analy_epoch = 'delay2',PSTH_log = PSTH_log)

### Plot sample neurons

In [ ]:
from analysis.PSTH_sample_neuron_by_trial import sample_neuron_by_trial
sample_neuron_by_trial(hp,log,model_dir,'odr','stim1',[616960,628480],'mix_neurons')
sample_neuron_by_trial(hp,log,model_dir,'odr','delay1',[616960,628480],'mix_neurons')
sample_neuron_by_trial(hp,log,model_dir,'odrd','stim1',[616960,628480],'mix_neurons')
sample_neuron_by_trial(hp,log,model_dir,'odrd','delay2',[616960,628480],'mix_neurons')

sample_neuron_by_trial(hp,log,model_dir,'odr','stim1',[616960,628480],'exh_neurons')
sample_neuron_by_trial(hp,log,model_dir,'odr','delay1',[616960,628480],'exh_neurons')
sample_neuron_by_trial(hp,log,model_dir,'odrd','stim1',[616960,628480],'exh_neurons')
sample_neuron_by_trial(hp,log,model_dir,'odrd','delay2',[616960,628480],'exh_neurons')

### Decoder
#### ODR

In [ ]:
from analysis.PSTH_Decoder_analysis import Decoder_analysis
Decoder_analysis(hp,model_dir,trial_num = log['trials'][-1],window_size=10,test_num=2)
Decoder_analysis(hp,model_dir,trial_num = log['trials'][452],window_size=10,test_num=2)#0.541015625
Decoder_analysis(hp,model_dir,trial_num = log['trials'][420],window_size=10,test_num=2)#0.3359375

#### ODRD

In [ ]:
from analysis.PSTH_Decoder_analysis import Decoder_analysis
Decoder_analysis(hp,model_dir,trial_num = log['trials'][-1],window_size=10,test_num=2,rule='odrd',)#0.95898
Decoder_analysis(hp,model_dir,trial_num = log['trials'][468],window_size=10,test_num=2,rule='odrd',)#0.54102
Decoder_analysis(hp,model_dir,trial_num = log['trials'][441],window_size=10,test_num=2,rule='odrd',)#0.35352

### Demixed PCA
#### ODR

In [ ]:
from analysis.PSTH_neuron_dPCA import neuron_dPCA
neuron_dPCA(hp,log,model_dir,trial_num = log['trials'][-1],rule='odr',task_mode='test',invert_tcomp_y=True,)
neuron_dPCA(hp,log,model_dir,trial_num = log['trials'][452],rule='odr',task_mode='test',invert_tcomp_y=True,)
neuron_dPCA(hp,log,model_dir,trial_num = log['trials'][420],rule='odr',task_mode='test',invert_tcomp_y=True,)

#### ODRD

In [ ]:
from analysis.PSTH_neuron_dPCA import neuron_dPCA
neuron_dPCA(hp,log,model_dir,trial_num = log['trials'][-1],rule='odrd',task_mode='test',appoint_loc_analysis=True, appoint_locs=[1,5],\
            invert_tcomp_y=True,invert_stcomp_y=True,tcomp_ylim=(-12,10),scomp_ylim=(-1,1),stcomp_ylim=(-15,15))
neuron_dPCA(hp,log,model_dir,trial_num = log['trials'][468],rule='odrd',task_mode='test',appoint_loc_analysis=True, appoint_locs=[1,5],\
            invert_tcomp_y=True,invert_stcomp_y=True,tcomp_ylim=(-12,10),scomp_ylim=(-1,1),stcomp_ylim=(-15,15))
neuron_dPCA(hp,log,model_dir,trial_num = log['trials'][441],rule='odrd',task_mode='test',appoint_loc_analysis=True, appoint_locs=[1,5],\
            invert_tcomp_y=True,invert_stcomp_y=True,tcomp_ylim=(-12,10),scomp_ylim=(-1,1),stcomp_ylim=(-15,15))

#### Overlap

In [ ]:
from analysis.PSTH_neuron_dPCA import neuron_dPCA
neuron_dPCA(hp,log,model_dir,trial_num = log['trials'][80],rule='overlap',task_mode='test',appoint_loc_analysis=True, appoint_locs=[0,4],\
            invert_scomp_y=True,invert_stcomp_y=True,tcomp_ylim=(-4,8),scomp_ylim=(-1.5,1.5),stcomp_ylim=(-3,3))#1.0
neuron_dPCA(hp,log,model_dir,trial_num = log['trials'][42],rule='overlap',task_mode='test',appoint_loc_analysis=True, appoint_locs=[1,5],\
            invert_scomp_y=True,invert_stcomp_y=True,tcomp_ylim=(-4,8),scomp_ylim=(-1.5,1.5),stcomp_ylim=(-3,3))#0.57422
neuron_dPCA(hp,log,model_dir,trial_num = log['trials'][29],rule='overlap',task_mode='test',appoint_loc_analysis=True, appoint_locs=[1,5],\
            invert_scomp_y=True,invert_stcomp_y=True,tcomp_ylim=(-4,8),scomp_ylim=(-1.5,1.5),stcomp_ylim=(-3,3))#0.32813